In [1]:
import librosa
import matplotlib.pyplot as plt
import random
import numpy as np
import os
import pandas as pd
import scipy


In [101]:
# load data
X = 'your own signal'
Sr = sr

mfccs = librosa.feature.mfcc(y = X, sr = Sr, n_mfcc = 4)

## Find variance MEL1 (first MFCC coefficient)

In [57]:
var_threshold_intact = []
var_threshold_error = []

for i in data_intact:
    mfccs = librosa.feature.mfcc(y = i, sr = Sr, n_mfcc = 4)
    mfccs_var = mfccs.var(axis=1)
    var_threshold_intact.append(mfccs_var[0])

for j in error_mix:
    mfccs = librosa.feature.mfcc(y = j, sr = Sr, n_mfcc = 4)
    mfccs_var = mfccs.var(axis=1)
    var_threshold_error.append(mfccs_var[0])
    
VTI = np.std(var_threshold_intact), VTE = np.std(var_threshold_error)

In [87]:
mfccs_var = mfccs.var(axis=1)
if mfccs_var[0] > 4500:
    print('Abnormal')

## Find zero variation

In [47]:
MEL1, MEL2, MEL3 = mfccs[0], mfccs[1], mfccs[2]
n = len(MEL1)

for i in range(n - 3):
    if all(MEL1[j+1] - MEL1[j] == 0 for j in range(i, i+3)) and \
       all(MEL2[j+1] - MEL2[j] == 0 for j in range(i, i+3)) and \
       all(MEL3[j+1] - MEL3[j] == 0 for j in range(i, i+3)):
        print('ERROR')
        break
else:
    print('intact')
               


ERROR


## Find long zero variation

In [83]:
def find_short_variation(mfccs):
    
    
    M1 = mfccs[0]
    M2 = mfccs[1]
    M3 = mfccs[2]

    threshold = 1
    
    diff_M1 = np.diff(M1)
    diff_M2 = np.diff(M2)
    diff_M3 = np.diff(M3)

    OT1 = np.where(np.abs(diff_M1) < threshold)[0]
    OT2 = np.where(np.abs(diff_M2) < threshold)[0]
    OT3 = np.where(np.abs(diff_M3) < threshold)[0]

    total_silent = OT1 & OT1 & OT1

    
    return(total_silent)

def group_consecutive_elements(arr):
    
    groups = []
    # Initialize the first group with the first element
    current_group = [arr[0]]

    # Iterate through the array starting from the second element
    for i in range(1, len(arr)):
        if arr[i] == arr[i-1] + 1:
            current_group.append(arr[i])
        else:
            # Add the current group to groups if it has at least 3 elements
            if len(current_group) >= 20:
                groups.append(current_group)
            current_group = [arr[i]]
    
    # Add the last group to groups if it has at least 3 elements
    if len(current_group) >= 20:
        groups.append(current_group)
    
    return groups

def number_of_groups(arr):
    grouped_elements = group_consecutive_elements(arr)
    return len(grouped_elements)



mfcc_jump = find_short_variation(mfccs)

grouped_elements = group_consecutive_elements(mfcc_jump)
num_groups = number_of_groups(mfcc_jump)
print(f"Grouped elements: {grouped_elements}")
print(f"Number of groups: {num_groups}")

Grouped elements: []
Number of groups: 0


In [84]:
if num_groups > 0:
    print('Abnormal')

## Find Flatness

In [93]:
flat_threshold_intact = []
flat_threshold_error = []

for i in data_intact:
    spectral_flatness = librosa.feature.spectral_flatness(y=i)[0]
    flat_threshold_intact.append(np.mean(spectral_flatness))
                                 
for j in error_mix:
    spectral_flatness = librosa.feature.spectral_flatness(y=j)[0]
    flat_threshold_error.append(np.mean(spectral_flatness))


    

In [99]:
np.std(flat_threshold_intact), np.std(flat_threshold_error)

(0.01937601, 0.050345194)

In [102]:
spectral_flatness = librosa.feature.spectral_flatness(y=X)[0]

if (np.mean(spectral_flatness) > 0.02):
    print('Abnormal')
    

Abnormal
